In [1]:
# install: pip install matplotlib

import math
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

# ---------- L-system generation ----------

def lsystem_iterate(axiom, rules, n, rng=None):
    if rng is None:
        import random
        rng = random

    s = axiom
    for _ in range(n):
        out = []
        for ch in s:
            r = rules.get(ch)
            if r is None:
                out.append(ch)
            elif isinstance(r, str):
                out.append(r)
            else:
                # stochastic: list of (p, replacement)
                x = rng.random()
                acc = 0.0
                for p, rep in r:
                    acc += p
                    if x <= acc:
                        out.append(rep)
                        break
        s = "".join(out)
    return s

# ---------- Turtle → segments ----------

def lsystem_turtle(s, step=5.0, angle_deg=25.0):
    angle = math.radians(angle_deg)
    x, y = 0.0, 0.0
    heading = math.pi / 2.0  # up
    stack = []
    segments = []

    for ch in s:
        if ch == "F":
            x2 = x + step * math.cos(heading)
            y2 = y + step * math.sin(heading)
            segments.append(((x, y), (x2, y2)))
            x, y = x2, y2
        elif ch == "f":
            x += step * math.cos(heading)
            y += step * math.sin(heading)
        elif ch == "+":
            heading += angle
        elif ch == "-":
            heading -= angle
        elif ch == "[":
            stack.append((x, y, heading))
        elif ch == "]":
            x, y, heading = stack.pop()
    return segments

# ---------- Plot high-res with Matplotlib ----------

def plot_segments(segments, filename="lsystem.png", dpi=600):
    lines = [(a, b) for a, b in segments]
    lc = LineCollection(lines, linewidths=0.6, colors="black")

    fig, ax = plt.subplots(figsize=(6, 6), dpi=dpi)
    ax.add_collection(lc)

    xs = [p[0] for seg in segments for p in seg]
    ys = [p[1] for seg in segments for p in seg]

    ax.set_aspect("equal", "box")
    ax.set_xlim(min(xs), max(xs))
    ax.set_ylim(min(ys), max(ys))
    ax.axis("off")

    fig.savefig(filename, bbox_inches="tight", pad_inches=0.05)
    plt.close(fig)

# ---------- Example: stochastic plant ----------

axiom = "X"
rules = {
    "X": [
        (0.33, "F-[[X]+X]+F[+FX]-X"),
        (0.33, "F[+X]-X"),
        (0.34, "F[-X]+X"),
    ],
    "F": "FF",
}

s = lsystem_iterate(axiom, rules, n=6)
segments = lsystem_turtle(s, step=4.0, angle_deg=22.5)
plot_segments(segments, "plant.png", dpi=600)


In [2]:
# symmetric, dense-ish tree (deterministic)
axiom = "X"
rules = {
    "X": "F[+X][-X]F[+X][-X]",  # symmetric left/right branching, duplicated
    "F": "FF",                 # densify by doubling forward segments
}
angle_deg = 20.0

s = lsystem_iterate(axiom, rules, n=6)
segments = lsystem_turtle(s, step=4.0, angle_deg=22.5)
plot_segments(segments, "plant__1.png", dpi=600)

In [118]:

axiom = "X"
rules = {
    "X": [
        (0.5, "F[+X][-X]F[+X][-X]"),      # perfectly symmetric
        (0.25, "F[+X]F[-X]F[+X][-X]"),    # a bit fuller on the right
        (0.25, "F[+X][-X]F[+X]F[-X]"),    # a bit fuller on the left
    ],
    "F": [
        (0.6, "FF"),   # dominant: dense
        (0.25, "FFF"), # occasionally extra dense
        (0.50, "F"),   # sometimes thinner
    ],
}
angle_deg = 20.0  


# plug these into the previous lsystem_iterate / lsystem_turtle code

num = 10

s = lsystem_iterate(axiom, rules, n=6)
segments = lsystem_turtle(s, step=20.0, angle_deg=angle_deg)
plot_segments(segments, f"lindenmayer_trees/plant__{num}.png", dpi=600)


In [ ]:

axiom = "X"
rules = {
    "X": [
        (0.5, "F[+X][-X]F[+X][-X]"),      # perfectly symmetric
        (0.25, "F[+X]F[-X]F[+X][-X]"),    # a bit fuller on the right
        (0.25, "F[+X][-X]F[+X]F[-X]"),    # a bit fuller on the left
    ],
    "F": [
        (0.6, "FF"),   # dominant: dense
        (0.25, "FFF"), # occasionally extra dense
        (0.50, "F"),   # sometimes thinner
    ],
}
angle_deg = 20.0  


# plug these into the previous lsystem_iterate / lsystem_turtle code

num = 10

s = lsystem_iterate(axiom, rules, n=6)
segments = lsystem_turtle(s, step=20.0, angle_deg=angle_deg)
plot_segments(segments, f"lindenmayer_trees/plant__{num}.png", dpi=600)
